In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-09-27"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
1170,2024-09-27,Ásia Wbl Asia Women,05:00,Surabaya Emdee Fever F,Fujitsu F,1.83,1.83,135.5,1.83,1.83,21.5,1.83,1.83,82ueCrKb,0.546448,0.546448,0.546448,0.546448,0.092896,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1171,2024-09-27,Ásia Wbl Asia Women,08:00,Cathay Life F,Sichuan F,1.80,1.86,141.5,1.80,1.86,25.5,1.85,1.81,GSr3AMlB,0.555556,0.537634,0.555556,0.537634,0.093190,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.023184,0.023184,0.015456,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1172,2024-09-27,Áustria Superliga,14:00,UBSC Graz,BBC Nord,1.03,10.50,154.5,1.80,1.86,-19.5,1.96,1.72,xKnUYiMq,0.970874,0.095238,0.555556,0.537634,0.066112,0.6,0.4,NaN,NaN,152.560,23.730200,0.155547,384.172,333.716129,0.868663,152.560,698.108,112.24,158.34,0.0,0.0,0.0,0.0,1.161544,0.023184,0.092231,-2.95,-0.590,-0.050847,0.563011,0.4,-0.163011,-5.00,-1.000,-9.500000,0.000000,0.0,0.000000
1173,2024-09-27,Eslovênia Liga Otp Banka,13:00,Ilirija,Rogaska,1.59,2.16,161.5,1.87,1.79,-4.5,2.02,1.67,I3ih3juA,0.628931,0.462963,0.534759,0.558659,0.091894,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.214960,0.030912,0.134139,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1174,2024-09-27,Espanha Primera Feb,15:45,Castello,Oviedo,1.73,1.94,158.5,1.83,1.83,-2.5,1.95,1.72,jo3JZjD4,0.578035,0.515464,0.546448,0.546448,0.093499,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.080922,0.000000,0.088629,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1175,2024-09-27,Espanha Primera Feb,15:45,Tizona Burgos,Palencia,2.23,1.55,171.5,1.84,1.82,2.5,1.95,1.72,UVEAyDrh,0.448430,0.645161,0.543478,0.549451,0.093592,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.254409,0.007728,0.088629,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1176,2024-09-27,Europa Liga Bnxt,15:30,Limburg,Den Helder,1.05,6.98,150.5,1.85,1.81,-15.5,1.95,1.72,nye4Zvan,0.952381,0.143266,0.540541,0.552486,0.095647,0.6,0.0,NaN,NaN,133.944,35.196138,0.262768,417.894,435.930162,1.043160,99.726,0.000,139.87,1188.00,0.0,1.0,0.0,0.0,1.044369,0.015456,0.088629,-2.71,-0.542,-0.092251,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1177,2024-09-27,Filipinas Copa Dos Governadores,06:00,Rain Or Shine Paint.,Magnolia Hotshots,1.51,2.49,211.5,1.87,1.79,-5.5,2.05,1.69,4jpaShkQ,0.662252,0.401606,0.534759,0.558659,0.063858,0.0,0.0,NaN,NaN,154.932,21.488818,0.138698,NaN,NaN,NaN,0.000,0.000,159.30,163.18,0.0,0.0,0.0,0.0,0.346482,0.030912,0.136128,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1178,2024-09-27,Filipinas Copa Dos Governadores,08:30,NLEX Road Warriors,TNT Tropang Giga,2.97,1.37,195.5,1.87,1.87,5.5,2.01,1.72,UD5d8Xcf,0.336700,0.729927,0.534759,0.534759,0.066627,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,297.67,0.00,1.0,0.0,0.0,0.0,0.521369,0.000000,0.109952,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1179,2024-09-27,França Lnb,15:00,La Rochelle,Cholet,2.86,1.43,148.5,1.89,1.90,4.5,2.01,1.78,lh5TNJ63,0.349650,0.699301,0.529101,0.526316,0.048951,0.0,0.4,NaN,NaN,106.870,17.112878,0.160128,163.906,46.7

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

Sem jogos
